In [1]:
import tensorflow as tf


In [ ]:
import tkinter as tk
from tkinter import filedialog
from PIL import ImageTk, Image
import tensorflow as tf
import numpy as np

# Parameters
batch_size = 32
img_height = 180
img_width = 180

# Load number of classes
test_ds = tf.keras.utils.image_dataset_from_directory(
    'data/Test data',
    image_size=(180, 180),
    batch_size=32
)

class_names = test_ds.class_names
print(class_names)
num_classes = len(class_names)

# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes)
])

# Load the model weights
model.load_weights('model4a.h5')

# Initialize GUI
root = tk.Tk()

# Function to load an image and make predictions
def predict_image():
    file_path = filedialog.askopenfilename()
    img = tf.keras.utils.load_img(file_path, target_size=(img_height, img_width))
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create a batch

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    print("This image most likely belongs to {} with a {:.2f} percent confidence.".format(class_names[np.argmax(score)], 100 * np.max(score)))

    image = Image.open(file_path).resize((img_width, img_height))
    image = image.convert("RGB")
    image = np.array(image) / 255.0
    image = np.expand_dims(image, axis=0)

    # Create a new window to display the results
    result_window = tk.Toplevel(root)
    result_window.title("Prediction Result")

    # Display the predicted class label
    class_label_text = "Predicted Class: {}".format(class_names[np.argmax(score)])
    class_label_label = tk.Label(result_window, text=class_label_text)
    class_label_label.pack()

    # Display the prediction confidence
    confidence_text = "Confidence: {:.2f}%".format(100 * np.max(score))
    confidence_label = tk.Label(result_window, text=confidence_text)
    confidence_label.pack()

    # Display the loaded image
    image_display = ImageTk.PhotoImage(Image.open(file_path).resize((300, 300)))
    image_label = tk.Label(result_window, image=image_display)
    image_label.image = image_display
    image_label.pack()

# Button to upload an image
upload_button = tk.Button(root, text="Upload Image", command=predict_image)
upload_button.pack()

# Run the GUI
root.mainloop()


Found 977 files belonging to 4 classes.
['Elephant', 'Panda', 'Tiger', 'Zebra']
1/1 [==============================] - 0s 333ms/step
This image most likely belongs to Zebra with a 99.38 percent confidence.


In [22]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

loss, accuracy = model.evaluate(test_ds)
print('Test accuracy:', accuracy)

23/23 [==============================] - 4s 162ms/step - loss: 0.2370 - accuracy: 0.9208
Test accuracy: 0.9207921028137207


In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 180, 180, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 180, 180, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 90, 90, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 90, 90, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 45, 45, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 45, 45, 64)        1